# Pandas数据处理——map、apply、applymap详解
在日常的数据处理中，经常会对一个DataFrame进行逐行、逐列和逐元素的操作，对应这些操作，Pandas中的map、apply和applymap可以解决绝大部分这样的数据处理需求。这篇文章就以案例附带图解的方式，为大家详细介绍一下这三个方法的实现原理，相信读完本文后，不论是小白还是Pandas的进阶学习者，都会对这三个方法有更深入的理解。

本文演示的数据集是模拟生成的，想练手的可以按下方的代码生成。

In [14]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import pandas as pd
boolean=[True,False]
gender=["男","女"]
color=["white","black","yellow"]
data=pd.DataFrame({
    "height":np.random.randint(150,190,100),
    "weight":np.random.randint(40,90,100),
    "smoker":[boolean[x] for x in np.random.randint(0,2,100)],
    "gender":[gender[x] for x in np.random.randint(0,2,100)],
    "age":np.random.randint(15,90,100),
    "color":[color[x] for x in np.random.randint(0,len(color),100) ]
}
)
data

,height,weight,smoker,gender,age,color
0,161,76,True,男,36,yellow
1,185,50,True,女,17,yellow
2,162,78,True,男,16,black
3,188,76,False,女,74,yellow
4,164,40,False,女,47,yellow
...,...,...,...,...,...,...
95,166,57,True,男,73,white
96,159,50,True,男,35,white
97,164,56,False,男,34,black
98,150,59,True,男,55,yellow


Series数据处理
1. map用法
如果需要把数据集中gender列的男替换为1，女替换为0，怎么做呢？绝对不是用for循环实现，使用Series.map()可以很容易做到，最少仅需一行代码。

In [15]:
#①使用字典进行映射
data["gender"] = data["gender"].map({"男":1, "女":0})
data
#②使用函数
# def gender_map(x):
#     gender = 1 if x == "男" else 0
#     return gender
# #注意这里传入的是函数名，不带括号
# data["gender"] = data["gender"].map(gender_map)
# data

,height,weight,smoker,gender,age,color
0,161,76,True,1,36,yellow
1,185,50,True,0,17,yellow
2,162,78,True,1,16,black
3,188,76,False,0,74,yellow
4,164,40,False,0,47,yellow
...,...,...,...,...,...,...
95,166,57,True,1,73,white
96,159,50,True,1,35,white
97,164,56,False,1,34,black
98,150,59,True,1,55,yellow


那map在实际过程中是怎么运行的呢？请看下面的图解（为了方便展示，仅截取了前10条数据）
![](https://klmtldh.oss-cn-shenzhen.aliyuncs.com/markdown/Pathon_pandas%E6%8C%89%E5%AD%97%E5%85%B8%E6%98%A0%E5%B0%84%E7%9A%84map%E5%8E%9F%E7%90%86.jpg)
![](https://klmtldh.oss-cn-shenzhen.aliyuncs.com/markdown/Pathon_pandas%E6%8C%89%E5%87%BD%E6%95%B0%E6%98%A0%E5%B0%84%E7%9A%84map%E5%8E%9F%E7%90%86.jpg)
不论是利用字典还是函数进行映射，map方法都是把对应的数据逐个当作参数传入到字典或函数中，得到映射后的值。
2. apply
同时Series对象还有apply方法，apply方法的作用原理和map方法类似，区别在于apply能够传入功能更为复杂的函数。怎么理解呢？一起看看下面的例子。

假设在数据统计的过程中，年龄age列有较大误差，需要对其进行调整（加上或减去一个值），由于这个加上或减去的值未知，故在定义函数时，需要加多一个参数bias，此时用map方法是操作不了的（传入map的函数只能接收一个参数），apply方法则可以解决这个问题。

In [16]:
def apply_age(x,bias):
    return x+bias

#以元组的方式传入额外的参数
data["age"] = data["age"].apply(apply_age,args=(-3,))
data

,height,weight,smoker,gender,age,color
0,161,76,True,1,33,yellow
1,185,50,True,0,14,yellow
2,162,78,True,1,13,black
3,188,76,False,0,71,yellow
4,164,40,False,0,44,yellow
...,...,...,...,...,...,...
95,166,57,True,1,70,white
96,159,50,True,1,32,white
97,164,56,False,1,31,black
98,150,59,True,1,52,yellow


可以看到age列都减了3，当然，这里只是简单举了个例子，当需要进行复杂处理时，更能体现apply的作用。

总而言之，对于Series而言，map可以解决绝大多数的数据处理需求，但如果需要使用较为复杂的函数，则需要用到apply方法。
DataFrame数据处理
1. apply
对DataFrame而言，apply是非常重要的数据处理方法，它可以接收各种各样的函数（Python内置的或自定义的），处理方式很灵活，下面通过几个例子来看看apply的具体使用及其原理。

在进行具体介绍之前，首先需要介绍一下DataFrame中axis的概念，在DataFrame对象的大多数方法中，都会有axis这个参数，它控制了你指定的操作是沿着0轴还是1轴进行。axis=0代表操作对列columns进行，axis=1代表操作对行row进行，如下图所示。
![](https://klmtldh.oss-cn-shenzhen.aliyuncs.com/markdown/Pathon_pandas_axis%E5%8F%82%E6%95%B0%E7%A4%BA%E4%BE%8B.jpg)
如果还不是很了解，没关系，下面会分别对apply沿着0轴以及1轴的操作进行讲解，继续往下走。

假设现在需要对data中的数值列分别进行取对数和求和的操作，这时可以用apply进行相应的操作，因为是对列进行操作，所以需要指定axis=0，使用下面的两行代码可以很轻松地解决我们的问题。

In [17]:
# 沿着0轴求和
data[["height","weight","age"]].apply(np.sum, axis=0)

# # 沿着0轴取对数
# data[["height","weight","age"]].apply(np.log, axis=0)

height    16934
weight     6538
age        5042
dtype: int64

实现的方式很简单，但调用apply时究竟发生了什么呢？过程是怎么实现的？还是通过图解的方式来一探究竟。（取前五条数据为例）
![](https://klmtldh.oss-cn-shenzhen.aliyuncs.com/markdown/Pathon_pandas_%E6%8C%89%E5%88%97%EF%BC%88axis=0%EF%BC%89apply%E6%B1%82%E5%92%8C%E5%AE%9E%E7%8E%B0%E8%BF%87%E7%A8%8B.jpg)
![](https://klmtldh.oss-cn-shenzhen.aliyuncs.com/markdown/Pathon_pandas_%E6%8C%89%E5%88%97%EF%BC%88axis=0%EF%BC%89apply%E5%8F%96%E5%AF%B9%E6%95%B0%E5%AE%9E%E7%8E%B0%E8%BF%87%E7%A8%8B.jpg)
当沿着轴0（axis=0）进行操作时，会将各列(columns)默认以Series的形式作为参数，传入到你指定的操作函数中，操作后合并并返回相应的结果。

那如果在实际使用中需要按行进行操作（axis=1）,那整个过程又是怎么实现的呢？

在数据集中，有身高和体重的数据，所以根据这个，我们可以计算每个人的BMI指数（体检时常用的指标，衡量人体肥胖程度和是否健康的重要标准），计算公式是：体重指数BMI=体重/身高的平方（国际单位kg/㎡），因为需要对每个样本进行操作，这里使用axis=1的apply进行操作，代码如下：

In [18]:
def BMI(series):
    weight = series["weight"]
    height = series["height"]/100
    BMI = weight/height**2
    return BMI

data["BMI"] = data.apply(BMI,axis=1)
data

,height,weight,smoker,gender,age,color,BMI
0,161,76,True,1,33,yellow,29.319856
1,185,50,True,0,14,yellow,14.609204
2,162,78,True,1,13,black,29.721079
3,188,76,False,0,71,yellow,21.502943
4,164,40,False,0,44,yellow,14.872100
...,...,...,...,...,...,...,...
95,166,57,True,1,70,white,20.685150
96,159,50,True,1,32,white,19.777699
97,164,56,False,1,31,black,20.820940
98,150,59,True,1,52,yellow,26.222222


还是用图解的方式来看看这个过程到底是怎么实现的（以前5条数据为例）。
![](https://klmtldh.oss-cn-shenzhen.aliyuncs.com/markdown/Pathon_pandas_%E6%8C%89%E8%A1%8C%EF%BC%88axis=1%EF%BC%89apply%E6%B1%82BMI%E6%8C%87%E6%95%B0%E5%AE%9E%E7%8E%B0%E8%BF%87%E7%A8%8B.jpg)
当apply设置了axis=1对行进行操作时，会默认将每一行数据以Series的形式（Series的索引为列名）传入指定函数，返回相应的结果。

总结一下对DataFrame的apply操作：

当axis=0时，对每列columns执行指定函数；当axis=1时，对每行row执行指定函数。
无论axis=0还是axis=1，其传入指定函数的默认形式均为Series，可以通过设置raw=True传入numpy数组。
对每个Series执行结果后，会将结果整合在一起返回（若想有返回值，定义函数时需要return相应的值）
当然，DataFrame的apply和Series的apply一样，也能接收更复杂的函数，如传入参数等，实现原理是一样的，具体用法详见官方文档。
2. applymap
applymap的用法比较简单，会对DataFrame中的每个单元格执行指定函数的操作，虽然用途不如apply广泛，但在某些场合下还是比较有用的，如下面这个例子。

为了演示的方便，新生成一个DataFrame

In [19]:
df = pd.DataFrame(
    {
        "A":np.random.randn(5),
        "B":np.random.randn(5),
        "C":np.random.randn(5),
        "D":np.random.randn(5),
        "E":np.random.randn(5),
    }
)
df

,A,B,C,D,E
0,-0.227157,-1.411041,-0.775066,1.598929,-0.878550
1,-3.130520,-1.182921,-0.626627,0.504265,-1.106215
2,-1.111041,0.960423,-0.268324,-1.722933,-1.049431
3,0.870037,0.279221,0.000426,1.830488,-1.503349
4,-1.090453,-0.375451,2.148521,0.703630,1.767414


现在想将DataFrame中所有的值保留两位小数显示，使用applymap可以很快达到你想要的目的，代码和图解如下：

In [20]:
df.applymap(lambda x:"%.2f" % x)

,A,B,C,D,E
0,-0.23,-1.41,-0.78,1.60,-0.88
1,-3.13,-1.18,-0.63,0.50,-1.11
2,-1.11,0.96,-0.27,-1.72,-1.05
3,0.87,0.28,0.00,1.83,-1.50
4,-1.09,-0.38,2.15,0.70,1.77


![](https://klmtldh.oss-cn-shenzhen.aliyuncs.com/markdown/Pathon_pandas_applymap%E5%AE%9E%E7%8E%B0%E8%BF%87%E7%A8%8B.jpg)